In [4]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import joblib
import json
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [7]:
model = keras.models.load_model('movie_recommendation_model.h5')

scaler = joblib.load('scaler.pkl')
min_max_scaler = joblib.load('min_max_scaler.pkl')
director_encoder = joblib.load('director_encoder.pkl')
title_tokenizer = joblib.load('title_tokenizer.pkl')
movies = pd.read_csv("neural_net_ready_preprocessed_movies.csv")

with open('actor_to_index.json', 'r') as f:
    actor_to_index = json.load(f)
all_actors = np.load('all_actors.npy', allow_pickle=True).tolist()
with open('genre_columns.json', 'r') as f:
    genre_columns = json.load(f)

if 'Director_Encoded' not in movies.columns:
     movies['Director_Encoded'] = director_encoder.transform(movies['Director'])

if 'Title_Sequences' not in movies.columns:
    sequences = title_tokenizer.texts_to_sequences(movies['Title'])
    padded_sequences = pad_sequences(sequences, maxlen=20)
    title_data = padded_sequences

if 'Star_Cast_Encoded' not in movies.columns:
     movies['Star Cast List Clean'] = movies['Star Cast List Clean'].apply(eval) 
     star_cast_encoded = np.zeros((len(movies), len(all_actors)))
     for i, actors in enumerate(movies['Star Cast List Clean']):
         for actor in actors:
             if actor in actor_to_index:
                 index = actor_to_index[actor]
                 star_cast_encoded[i, index] = 1

numerical_data = movies[['IMDb Rating', 'Duration (minutes)', 'Year']].values
director_data = movies['Director_Encoded'].values 
genre_data = movies[genre_columns].values

num_movies = len(movies_df)
generic_user_id = 0 